#  Notebook using Azur SDK.NET

In [ ]:
#r "nuget: Azure.ResourceManager.MachineLearning, 1.0.0-beta.1"
#r "nuget: Azure.Identity, 1.6.0"
#r "nuget: Azure.Security.KeyVault.Secrets, 4.3.0"
#r "nuget:Azure.Storage.Blobs, 12.12.0"
#r "Microsoft.VisualBasic"

#r "nuget: Microsoft.ML, 2.0.0-preview.22314.3"
#r "nuget: Microsoft.ML.AutoML, 0.20.0-preview.22314.3"
#r "nuget: Newtonsoft.Json, 13.0.2-beta1"

Installed Packages Azure.Identity, 1.6.0 Azure.ResourceManager.MachineLearning, 1.0.0-beta.1 Azure.Security.KeyVault.Secrets, 4.3.0 Azure.Storage.Blobs, 12.12.0 Microsoft.ML, 2.0.0-preview.22314.3 Microsoft.ML.AutoML, 0.20.0-preview.22314.3 Newtonsoft.Json, 13.0.2-beta1

In [ ]:
// Import common usings.
using Azure.Storage.Blobs;
using Azure.Storage.Blobs.Models;
using System.IO;
using Microsoft.VisualBasic;
using Microsoft.VisualBasic.FileIO;



### The following section define file paths, azure resources and other utilities to complete this machine learning opration in this notebook.

In [ ]:
// Generate job id.
public string RandomString(int size, bool lowerCase)
        {
            StringBuilder builder = new StringBuilder();
            Random random = new Random();
            char ch;
            for (int i = 0; i < size; i++)
            {
                ch = Convert.ToChar(Convert.ToInt32(Math.Floor(26 * random.NextDouble() + 65)));
                builder.Append(ch);
            }
            if (lowerCase)
                return builder.ToString().ToLower();
            return builder.ToString();
        }



// File paths.
var imageFolderPath=@"C:\Users\zehailem\Desktop\AugustSunnyImages";
var sampleDataPath= @"C:\Users\zehailem\OneDrive - Microsoft\Testdata\AugustWeather\Clouady\cloudy1.jpg";
var selectedFolder = new DirectoryInfo(imageFolderPath).Name;
var tempPath = Path.GetTempPath();
var guid = Guid.NewGuid().ToString();
var modelName="MlModel";
var tsvFilePath = Path.Combine(tempPath, $"{guid}-{modelName}.ImageTrainData.tsv");
var jsonlFilePath = Path.Combine(tempPath, $"{guid}-{modelName}.AzureImageTrainData.jsonl");
var MlTableFolder = Directory.CreateDirectory(Path.Combine(tempPath, "TrainMlTable"));
var mLTableFilePath = Path.Combine(MlTableFolder.FullName, "MLTable.");
 static string labelFilePath= @"C:\Users\zehailem\Desktop\Notebook\Artifact\lables.json";
var onnxFilePath= @"C:\Users\zehailem\Desktop\Artifact\Notebook\onnxmodel.onnx";
static string modelFilePath = @"C:\Users\zehailem\Desktop\Artifact\\NotebookModel.zip";

// Azure resources Names.
string subscriptionName ="ML.NET Model Builder Demo";
string resourceGroupName = "ZewdPortlaExcercise";
string workspaceName ="BikeRentalWorkSpace";
string computeName ="AugustComp";
var experimentName = "ImageClassificationNotebook";
 string jobId =RandomString(15, true);


# Generate data schema for training 
The following  cells are to generate  data schemas appropriate for Image classification. For furthere detail refer this  https://docs.microsoft.com/en-us/azure/machine-learning/reference-automl-images-schema

## Generate tsv file

In [ ]:
public async Task WriteAllDataToTSVAsync(string selectedFolderName, string tsvFilePath)
        {
            DirectoryInfo rootDirectoryInfo = new DirectoryInfo(selectedFolderName);
            DirectoryInfo[] subDirectories = rootDirectoryInfo.GetDirectories();
			
             string[] AllowedImageFileExtensions = new[] { ".png", ".jpg", ".jpeg", ".gif" };
            using (var outFile = File.CreateText(tsvFilePath))
            {
                try
                {
                    // Add labels for the data.
                     string label = "Label";
                    
                    var labels = new string[] { label, "ImageSource" };
                    var headerString = string.Join("\t", labels);
                    outFile.WriteLine(headerString);

                    foreach (DirectoryInfo directory in subDirectories)
                    {
						TextFieldParser parser;
                        IEnumerable<FileInfo> files = directory.EnumerateFiles();
                        var folderList = files.Where(f => AllowedImageFileExtensions.Contains(f.Extension.ToLower()));
                            foreach (FileInfo file in folderList)
                            {
                                var values = new string[] { directory.Name, file.FullName };
                                var line = string.Join("\t", values);

                                outFile.WriteLine(line);
                            }
                    }
                        
                    
                }
                catch (Exception e)
                {
                    throw new Exception();
                }
            }

            
        }

## Generate MLTable File

In [ ]:

        public void CreateMlTableFile(string selectedFolderName, string mlTableFilePath, string remoteAzureTrainingFile)
        {

            try
            {
                using (var outFile = File.CreateText(mlTableFilePath))
                {
                    var jsonlFileName = Path.GetFileName(remoteAzureTrainingFile);
                    var selectedFolder = new DirectoryInfo(selectedFolderName).Name;
                    var mlTableContent = $@"paths:
  - file: azureml://datastores/workspaceblobstore/paths/{selectedFolder}/{jsonlFileName}
transformations:
  - read_json_lines:
        encoding: utf8
        invalid_lines: error
        include_path_column: false
  - convert_column_types:
      - columns: image_url
        column_type: stream_info";
                    outFile.Write(mlTableContent);
                }
            }
            catch (Exception e)
            {
                throw new Exception(e.Message);

            }
        }
    

## Azure storage setup

### Configure your storage connection string
 https://docs.microsoft.com/en-us/azure/storage/blobs/storage-quickstart-blobs-dotnet?tabs=environment-variable-windows#configure-your-storage-connection-string

In [ ]:
  string connectionString = Environment.GetEnvironmentVariable("AZURE_STORAGE_CONNECTION_STRING");
  
  //Create a BlobServiceClient object which will be used to create a container client
  BlobServiceClient blobServiceClient = new BlobServiceClient(connectionString);
  var allblobs = blobServiceClient.GetBlobContainers(BlobContainerTraits.Metadata, BlobContainerStates.System);

  // Get workspaceblobstore.
  var defaultBlob= allblobs.First(b=> b.Name.StartsWith("azureml-blobstore")).Name;
  var defaultContainer = blobServiceClient.GetBlobContainerClient(defaultBlob);

  // This is the storage where artifacts stored.
  var artifactBlob= allblobs.First(b=> b.Name.Equals("azureml")).Name;
  var artifactBlobContainer = blobServiceClient.GetBlobContainerClient(artifactBlob);

###  Login into Azure  in powershell

In [ ]:
#!pwsh
az login


[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
    "id": "2f96ae42-240b-4228-bafa-26d8b7b03bf3",
    "isDefault": false,
    "managedByTenants": [],
    "name": "MSFT-CloudMS-CPT-PRD-01",
    "state": "Enabled",
    "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
    "user": {
      "name": "zehailem@microsoft.com",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
    "id": "8c4b5b03-3b24-4ed0-91f5-a703cd91b412",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Cosmos_C&E_Azure_AzureEngineeringSystems_100200",
    "state": "Enabled",
    "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
    "user": {
      "name": "zehailem@microsoft.com",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
    "id": "40dd91f6-ead1-40dd-a291-2ae37717981b",
    "isDefault": false,
  

# Use Azure SDKs to call into AzureML

In [ ]:
using Azure.ResourceManager;
using Azure.ResourceManager.MachineLearning;
using System;
using System.Threading.Tasks;
using Azure.Core;
using Azure.ResourceManager;
using Azure.ResourceManager.Resources;
using Azure.Identity;
using Azure.ResourceManager.MachineLearning.Models;
using Azure;

In [ ]:
var armClient = new ArmClient(new DefaultAzureCredential());
var subscription = armClient.GetSubscriptions();

## Select your subscription where you want to have machine learning oprations.

In [ ]:
var selectedSubscription = subscription.Where(sub => sub.Data.DisplayName == subscriptionName).First();
var subId= selectedSubscription.Data.SubscriptionId;

## Select your  resource group.

In [ ]:
var resourceGroups = selectedSubscription.GetResourceGroups();
var rg= resourceGroups.Where(r=> r.Data.Name == resourceGroupName).First();


## Select your workspace 

In [ ]:
MachineLearningWorkspaceResource selectedWorkspace = await rg.GetMachineLearningWorkspaceAsync(workspaceName);


Generate jsonl file

In [ ]:
 public async Task CreateJsonlFileForRemoteAsync(string imageTsvPath, string generatedTsvPath)
        {

            using (var outFile = File.CreateText(generatedTsvPath))
            {
                try
                {
                    using (TextFieldParser parser = new TextFieldParser(imageTsvPath))
                    {
                        parser.TextFieldType = FieldType.Delimited;
                        parser.SetDelimiters("\t");

                        // Skip the header line
                        parser.ReadLine();
                        while (!parser.EndOfData)
                        {
                            var row = parser.ReadFields();

                            var label = row[0];
                            var file = new FileInfo(row[1]);
                            var imageRelativePath = Path.Combine(file.Directory.Parent.Name, file.Directory.Name, file.Name).Replace('\\', '/');

                            var values = new string[] { imageRelativePath, label };

                            var jsonLine = $@"{{""image_url"":""azureml://subscriptions/{subId}/resourcegroups/{rg.Data.Name}/workspaces/{selectedWorkspace.Data.Name}/datastores/workspaceblobstore/paths/{imageRelativePath}"", ""label"":""{label}""}}";

                            outFile.WriteLine(jsonLine);
                        }
                    }
                }
                catch (Exception e)
                {
                    throw new Exception(e.Message);
                }
            }
        }

In [ ]:
public async Task UploadImageToAzureAsync(string imageSourceFolder, string remoteInputFile)
        {
			
            var localFolderName = Path.GetFileName(imageSourceFolder);

            using (TextFieldParser parser = new TextFieldParser(remoteInputFile))
            {
                while (!parser.EndOfData)
                {
                    var row = parser.ReadLine();
                    var urlLength = row.IndexOf(',');
                    var urlText = $@"{{""image_url"":""azureml://subscriptions/{subId}/resourcegroups/{rg.Data.Name}/workspaces/{selectedWorkspace.Data.Name}/datastores/workspaceblobstore/paths/{localFolderName}""";

                    // save path relative to localFolderSource.Path
                    var relativeFilePath = row.Substring(urlText.Length, urlLength - urlText.Length - 1);
                    var relativeImagePath = relativeFilePath.Replace('/', '\\');

                    var fullImagePath = Path.Combine(imageSourceFolder, relativeImagePath);
                    BlobClient blobClientImage = defaultContainer.GetBlobClient("./" + Path.Combine(localFolderName ,relativeImagePath));
                    await blobClientImage.UploadAsync(fullImagePath, true);
                }
            }
        }
			
    

In [ ]:
await WriteAllDataToTSVAsync(imageFolderPath,tsvFilePath);
await CreateJsonlFileForRemoteAsync(tsvFilePath,jsonlFilePath);
CreateMlTableFile(imageFolderPath,mLTableFilePath, jsonlFilePath);


In [ ]:
    //Upload jsonl file.
    BlobClient blobClient = defaultContainer.GetBlobClient("./" + selectedFolder +"/" + Path.GetFileName(jsonlFilePath));
	await blobClient.UploadAsync(jsonlFilePath , true);

	// upload MLTable file.
	BlobClient blobClientMlTable =defaultContainer.GetBlobClient("./" + selectedFolder +"/" +"TrainMlTable/" + Path.GetFileName(mLTableFilePath));
	await blobClientMlTable.UploadAsync(mLTableFilePath, true);
      
	//Upload images
	  await UploadImageToAzureAsync(imageFolderPath,jsonlFilePath);
	

## Start AutoML run

In [ ]:
public async Task<MachineLearningJobResource> SubmitAutoMLImageClassificationAsync(
            ResourceGroupResource resourceGroup,
            string workspaceName,
            string id,
            string experimentName,
            string computeId)
        {
            MachineLearningWorkspaceResource ws = await resourceGroup.GetMachineLearningWorkspaces().GetAsync(workspaceName);

            // Upload the MLTable in the default workspaceblobstore.
            var trainData = new MLTableJobInput(new Uri($"azureml://datastores/workspaceblobstore/paths/{selectedFolder}/TrainMlTable"))
            {
                Mode = InputDeliveryMode.EvalMount,
                Description = "Train data",
            };

            var trainingData = new TrainingDataSettings(trainData);

            ImageVerticalDataSettings dataSettings = new ImageVerticalDataSettings("label", trainingData);
            
            ImageLimitSettings limitSettings = new ImageLimitSettings()
            {
               
                Timeout = TimeSpan.FromHours(168)
            };

            

            AutoMLVertical taskDetails = new ImageClassification(dataSettings, limitSettings)
            {
                LogVerbosity = LogVerbosity.Info,
                PrimaryMetric = ClassificationPrimaryMetrics.Accuracy,
                
            };

            var autoMLJob = new AutoMLJob(taskDetails)
            {
                ExperimentName = experimentName,
                DisplayName = "AutoMLJob ImageClassification-" + Guid.NewGuid().ToString("n").Substring(0, 6),
                IsArchived = false,
                ComputeId = computeId,
                Resources = new ResourceConfiguration
                {
                    InstanceCount = 3,
                },
                Properties = new Dictionary<string, string>
                    {
                        { "property-name", "property-value" },
                    },
                Tags = new Dictionary<string, string>
                    {
                        { "tag-name", "tag-value" },
                    },
                EnvironmentVariables = new Dictionary<string, string>()
                    {
                        { "env-var", "env-var-value" }
                    },
                Description = "This is a description of test AutoMLJob for multi-class Image classification job using fridge items dataset",
            };

            MachineLearningJobData MachineLearningJobData = new MachineLearningJobData(autoMLJob);
            ArmOperation<MachineLearningJobResource> jobOperation = await ws.GetMachineLearningJobs().CreateOrUpdateAsync(WaitUntil.Completed, id, MachineLearningJobData);
            MachineLearningJobResource jobResource = jobOperation.Value;
           
       
            return jobResource;
        }

In [ ]:

var computeId = $"/subscriptions/{subId}/resourceGroups/{rg.Data.Name}/providers/Microsoft.MachineLearningServices/workspaces/{selectedWorkspace.Data.Name}/computes/{computeName}";

MachineLearningJobResource job=await SubmitAutoMLImageClassificationAsync(rg, selectedWorkspace.Data.Name, jobId, experimentName, computeId);


## Monitor remote run

In [ ]:
private static async Task WaitForJobToFinishAsync(
            MachineLearningWorkspaceResource ws,
            string id)
        {
            // delay between each retry (in milliseconds)
            const int SleepIntervalMs = 20 * 1000;
            MachineLearningJobResource jobResource = null;
            Console.WriteLine($"Starting to poll the status of Job Id: {id}");
            do
            {
                jobResource = await ws.GetMachineLearningJobs().GetAsync(id);
                Console.WriteLine($"DateTime: {DateTime.Now}, Experiment Name:'{jobResource.Data.Properties.ExperimentName}' status returned: '{jobResource.Data.Properties.Status}'.");

                if (jobResource.Data.Properties.Status != JobStatus.Completed && jobResource.Data.Properties.Status != JobStatus.Failed && jobResource.Data.Properties.Status != JobStatus.Canceled)
                {
                    await Task
                        .Delay(SleepIntervalMs)
                        .ConfigureAwait(false);
                }
            }
            while (jobResource.Data.Properties.Status != JobStatus.Completed && jobResource.Data.Properties.Status != JobStatus.Failed && jobResource.Data.Properties.Status != JobStatus.Canceled);

        }
        await WaitForJobToFinishAsync(selectedWorkspace, jobId);

Starting to poll the status of Job Id: pihtlntjivfjzcb
DateTime: 8/24/2022 11:49:11 PM, Experiment Name:'ImageClassificationNotebook' status returned: 'NotStarted'.
DateTime: 8/24/2022 11:49:31 PM, Experiment Name:'ImageClassificationNotebook' status returned: 'Running'.
DateTime: 8/24/2022 11:49:51 PM, Experiment Name:'ImageClassificationNotebook' status returned: 'Running'.
DateTime: 8/24/2022 11:50:11 PM, Experiment Name:'ImageClassificationNotebook' status returned: 'Running'.
DateTime: 8/24/2022 11:50:32 PM, Experiment Name:'ImageClassificationNotebook' status returned: 'Running'.
DateTime: 8/24/2022 11:50:52 PM, Experiment Name:'ImageClassificationNotebook' status returned: 'Running'.
DateTime: 8/24/2022 11:51:12 PM, Experiment Name:'ImageClassificationNotebook' status returned: 'Running'.
DateTime: 8/24/2022 11:51:32 PM, Experiment Name:'ImageClassificationNotebook' status returned: 'Running'.
DateTime: 8/24/2022 11:51:53 PM, Experiment Name:'ImageClassificationNotebook' status 

# Downlaod artifacts 

In [ ]:
var labelArtifact= artifactBlobContainer.GetBlobClient($"./ExperimentRun/dcid.{job.Data.Name}_HD_0/train_artifacts/labels.json");
var onnxModelArtifact= artifactBlobContainer.GetBlobClient($"./ExperimentRun/dcid.{job.Data.Name}_HD_0/train_artifacts/model.onnx");

await labelArtifact.DownloadToAsync(labelFilePath);
await onnxModelArtifact.DownloadToAsync(onnxFilePath);

Error: System.IO.DirectoryNotFoundException: Could not find a part of the path 'C:\Users\zehailem\Desktop\Notebook\Artifact\lables.json'.
   at Microsoft.Win32.SafeHandles.SafeFileHandle.CreateFile(String fullPath, FileMode mode, FileAccess access, FileShare share, FileOptions options)
   at Microsoft.Win32.SafeHandles.SafeFileHandle.Open(String fullPath, FileMode mode, FileAccess access, FileShare share, FileOptions options, Int64 preallocationSize)
   at System.IO.Strategies.OSFileStreamStrategy..ctor(String path, FileMode mode, FileAccess access, FileShare share, FileOptions options, Int64 preallocationSize)
   at System.IO.Strategies.FileStreamHelpers.ChooseStrategy(FileStream fileStream, String path, FileMode mode, FileAccess access, FileShare share, Int32 bufferSize, FileOptions options, Int64 preallocationSize)
   at System.IO.File.Create(String path)
   at Azure.Storage.Blobs.Specialized.BlobBaseClient.DownloadToAsync(String path, BlobRequestConditions conditions, Storage

# Consume Model

In [ ]:
using Microsoft.ML.Transforms.Image;
using System.Drawing;
using Microsoft.ML.Data;
using Microsoft.ML;
using Newtonsoft.Json;
using Microsoft.ML.AutoML.CodeGen;

### Create input and output models

In [ ]:
public class ModelInput
        {
            [ColumnName(@"Label")]
            public string Label { get; set; }

            [ColumnName(@"ImageSource")]
            [Microsoft.ML.Transforms.Image.ImageType(224, 224)]
            public System.Drawing.Bitmap ImageSource { get; set; }

        }

In [ ]:


// Get lebels map.
static string json= File.ReadAllText(labelFilePath, Encoding.UTF8);
static string[] labels = JsonConvert.DeserializeObject<string[]>(json);
 
 public class ModelOutput
        {
            [ColumnName("output1")]
            public float[] Output1 { get; set; }

            public string[] ClassificationLabels = labels;

            public string Prediction
            {
                get
                {
                    var maxScore = this.Score.Max();
                    var maxIndex = Array.IndexOf(this.Score, maxScore);
                    return this.ClassificationLabels[maxIndex];
                }
            }

            public float[] Score
            {
                get
                {
                    var exp = this.Output1.Select(x => (float)Math.Exp(x));
                    var exp_sum = exp.Sum();
                    return exp.Select(x => x / exp_sum).ToArray();
                }
            }
        }


In [ ]:
var mlContext = new MLContext();

In [ ]:

public class ImageClassificationBitmapModelInput
{
    [ColumnName("Label")]
    [LoadColumn(0)]
    public string Label { get; set; }

    [ColumnName("ImageSource"), LoadColumn(1), ImageType(224, 224)]
    public Bitmap ImageSource { get; set; }
}

 
 var pipeline = mlContext.Transforms.ResizeImages("ImageSource", 224, 224, "ImageSource")
                             .Append(mlContext.Transforms.ExtractPixels("input1", "ImageSource"))
                             .Append(mlContext.Transforms.ApplyOnnxModel(modelFile: onnxFilePath));

            var trainData = mlContext.Data.LoadFromEnumerable(new[] { new ImageClassificationBitmapModelInput() });
            var bestModel = pipeline.Fit(trainData);

			 
 
        

In [ ]:
mlContext.Model.Save(bestModel,trainData.Schema,modelFilePath);

# Create PredictionEngine

In [ ]:
 public PredictionEngine<ModelInput, ModelOutput> CreatePredictEngine()
 {
	 ITransformer mlModel = mlContext.Model.Load(modelFilePath, out var _);
	 return  mlContext.Model.CreatePredictionEngine<ModelInput, ModelOutput>(mlModel);
  }
 
readonly Lazy<PredictionEngine<ModelInput, ModelOutput>> PredictEngine = new Lazy<PredictionEngine<ModelInput, ModelOutput>>(() => CreatePredictEngine(), true);
 var predEngine = PredictEngine.Value;
 

## Try it out

In [ ]:
var image = (Bitmap)System.Drawing.Image.FromFile(sampleDataPath);


ModelInput sampleInput= new ModelInput()
{
  ImageSource = image,
};
ModelOutput predictionResult = predEngine.Predict(sampleInput);
 Console.WriteLine($"\n\nPredicted Label value: {predictionResult.Prediction} \nPredicted Label scores: [{String.Join(",", predictionResult.Score)}]\n\n");



Predicted Label value: sunny 
Predicted Label scores: [NaN,0]


